### Ejercicio

- Se requiere obtener información sobre el “total del presupuesto” y “total de ingresos” de las
“películas” donde el “año de la fecha de lanzamiento” debe ser mayor o igual a 2015,
también debe estar agrupado por el “año de la fecha de lanzamiento” y el “país” en donde
se realizó la película.

- También se requiere realizar un ranking ordenado de manera “descendente” por el “total
del presupuesto” y “total de ingresos” particionado por el “Año de la fecha de lanzamiento”.

#### Leer todos los datos que son requeridos

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
#movie_df = spark.read.parquet(f"{silver_folder_path}/movies") \
#            .filter(f"file_date = '{v_file_date}'")

#Modulo: Delta Lake

movie_df = spark.read.format("delta").load(f"{silver_folder_path}/movies") \
            .filter(f"file_date = '{v_file_date}'")

In [0]:
#country_df = spark.read.parquet(f"{silver_folder_path}/countries")

#Modulo: Delta Lake

country_df = spark.read.format("delta").load(f"{silver_folder_path}/countries")

In [0]:
#production_country_df = spark.read.parquet(f"{silver_folder_path}/productions_countries") \
#            .filter(f"file_date = '{v_file_date}'")

#Modulo: Delta Lake

production_country_df = spark.read.format("delta").load(f"{silver_folder_path}/productions_countries") \
            .filter(f"file_date = '{v_file_date}'")

### Join "country" y "production_country"

In [0]:
country_prod_coun_df = country_df.join(production_country_df,
                                         country_df.country_id == production_country_df.country_id,
                                         "inner") \
                        .select(country_df.country_name, production_country_df.movie_id)

### Join "movies_df" y "country_prod_coun_df"

%md
- Filtrar las peliculas donde su fecha de lanzamiento sea mayor o igual a 2015

In [0]:
movies_filter_df = movie_df.filter(movie_df.year_release_date >= 2015)

In [0]:
results_movies_country_prod_df = movies_filter_df.join(country_prod_coun_df,
                                                     movies_filter_df.movie_id == country_prod_coun_df.movie_id,
                                                     "inner")

In [0]:
results_df = results_movies_country_prod_df.select("year_release_date","budget","revenue","country_name")

In [0]:
# Agrupamos por year_release_date, country_name
from pyspark.sql.functions import sum
results_group_by_df = results_df.groupBy("year_release_date","country_name") \
                                      .agg(
                                          sum("budget").alias("tota_budget"),
                                          sum("revenue").alias("total_revenue")
                                      ) 

In [0]:
from pyspark.sql.functions import lit

In [0]:
# Realizar la particion
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, dense_rank, asc

results_dense_rank_df = Window.partitionBy("year_release_date").orderBy(
                                                              desc("tota_budget"),
                                                              desc("total_revenue")
                                                              )

final_df = results_group_by_df.withColumn("dense_rank", dense_rank().over(results_dense_rank_df)) \
                              .withColumn("created_date", lit(v_file_date))

### Escribir en el DataLake en formato "Delta"

In [0]:
#overwrite_partition(final_df,"movie_gold","results_group_movie_country","created_date")

In [0]:
# final_df.write.mode("overwrite").parquet(f"{gold_folder_path}/results_group_movie_country")

# Crear una tabla en base al data frame final_df
# Creara la tabla results_group_movie_country en a DB que se llame movie_gold
# Modulo: Spark SQL - Database / Table / View: seccion: Crear Tabla - Capa Gold
#final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_gold.results_group_movie_country")

#Nota: Esto es una tabla administrada por Databricks, esto quiere decir que si la tabla se elimina, los registros tambien se van a eliminar
# Append, agrega los registros a la tabla existente, carga incremental de datos

#final_df.write.mode("append").partitionBy("created_date").format("parquet").saveAsTable("movie_gold.results_group_movie_country")

#Modulo: Delta Lake
merge_condition = "tgt.year_release_date = src.year_release_date AND tgt.country_name = src.country_name AND tgt.created_date = src.created_date" # Buscar por particion
merge_delta_lake(final_df,"movie_gold","results_group_movie_country",gold_folder_path,merge_condition,"created_date")

In [0]:
%sql
-- Consultar la DB movie_gold, tabla results_group_movie_country
SELECT created_date, COUNT(1) 
FROM movie_gold.results_group_movie_country
GROUP BY 1;

In [0]:
#display(spark.read.parquet(f"{gold_folder_path}/results_group_movie_country"))
display(spark.read.format("delta").load(f"{gold_folder_path}/results_group_movie_country"))